<a href="https://colab.research.google.com/github/keshavkumar03/machine_learning-web-app/blob/main/generate_sql_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Query Generator 🤖🧑🏻‍💻
This notebook demonstrates the ability to generate SQL queries from a given natural language question. The notebook is divided into two parts:
1. **Data Generation**: This section generates fake CRM data and stores it in a SQL database.
2. **Query Generation**: This section generates SQL queries from natural language questions.

## Data Generation
The following code generates fake CRM data for a second hand car market and stores it in a SQL database. The data is generated using the [Faker](https://faker.readthedocs.io/en/master/) library.

In [ ]:
from faker import Faker
from faker_vehicle import VehicleProvider
import pandas as pd
from sqlalchemy import create_engine, text
import os

server = os.environ['SQL_DATABASE_HOST']
database = os.environ['SQL_DATABASE_NAME']
username = os.environ['SQL_DATABASE_USER']
password = os.environ['SQL_DATABASE_PWD']
driver = '{ODBC Driver 17 for SQL Server}'

# Create a connection string
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# use sqlalchemy to create a connection to the database
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={connection_string}")

In [ ]:
# Create a Faker instance for Belgium
fake = Faker('nl_BE')
fake.add_provider(VehicleProvider)

def generate_customer_data(n):
    """Generate n rows of fake customer data."""
    data = []
    for _ in range(n):
        data.append([fake.unique.random_number(digits=5),
                     fake.first_name(),
                     fake.last_name(),
                     fake.email(),
                     fake.phone_number(),
                     fake.street_address(),
                     fake.city(),
                     fake.postcode(),
                     'Belgium'])
    return data


# Generate 10K rows of data
data = generate_customer_data(10000)

# Create a pandas DataFrame
df = pd.DataFrame(data, columns=['CustomerID', 'FirstName', 'LastName', 'Email', 'PhoneNumber', 'Address', 'City', 'PostalCode', 'Country'])

# Save the data from dataframe to SQL Server, create a connection to the database
with engine.connect() as conn:
    df.to_sql('customers', conn, if_exists='replace', index=False)

In [ ]:
# Now let's generate a table of 100 cars: productID, brand, model, year, price
fake.unique.clear()
def generate_product_data(n):
    """Generate n rows of fake product data."""
    data = []
    for _ in range(n):
        vehicle = fake.vehicle_object()
        data.append([fake.unique.random_number(digits=5),
                     vehicle['Make'],
                     vehicle['Model'],
                     vehicle['Year'],
                     fake.pydecimal(left_digits=5, right_digits=2, positive=True, min_value=100, max_value=10000)])
    return data

# Generate 100 rows of data
data = generate_product_data(100)

# Store in the database
df = pd.DataFrame(data, columns=['ProductID', 'Brand', 'Model', 'Year', 'Price'])
with engine.connect() as conn:
    df.to_sql('cars', conn, if_exists='replace', index=False)

In [ ]:
# Now let's finally generate a table of 100K carsales data: SalesID, CustomerID, ProductID, Quantity, Price, DiscountPercent, Total, SalesAgent, Date
fake.unique.clear()


def generate_sales_data(n):
    """Generate n rows of fake sales data."""
    cars = pd.read_sql('SELECT ProductID, Price FROM cars', engine)
    customer_ids = pd.read_sql('SELECT CustomerID FROM customers', engine)
    data = []
    for _ in range(n):
        car = cars.sample().iloc[0]
        quantity = fake.random_int(min=1, max=10)
        discount = fake.random_int(min=0, max=10)
        data.append([fake.unique.random_number(digits=5),
                     customer_ids.sample().iloc[0]['CustomerID'],
                     car['ProductID'],
                     quantity,
                     car['Price'],
                     fake.random_int(min=0, max=10),
                     float(car['Price']) * quantity * (1 - discount/100),
                     fake.name(),
                     fake.date_between(start_date='-1y', end_date='today')])
    return data

# Generate 10K rows of data
data = generate_sales_data(10000)

# Store in the database
df = pd.DataFrame(data, columns=['SalesID', 'CustomerID', 'ProductID', 'Quantity', 'Price', 'DiscountPercent', 'Total', 'SalesAgent', 'Date'])
with engine.connect() as conn:
    df.to_sql('carsales', conn, if_exists='replace', index=False)


### Let's write some SQL queries to get some insights from the data
After this we will use langchain to visualize the data and generate SQL queries from natural language questions.

In [ ]:
conn = engine.connect()
# Display the first 5 rows of the customers table
print(pd.read_sql('SELECT TOP 5 * FROM customers', conn).to_markdown())

|    |   CustomerID | FirstName   | LastName   | Email                 | PhoneNumber   | Address           | City                   |   PostalCode | Country   |
|---:|-------------:|:------------|:-----------|:----------------------|:--------------|:------------------|:-----------------------|-------------:|:----------|
|  0 |        28485 | Lucas       | De Wilde   | opeeters@example.com  | (0647) 305383 | Jozefpad 421      | Ramskapelle-Nieuwpoort |         9903 | Belgium   |
|  1 |        53848 | Benjamin    | Vermeiren  | yvonne97@example.net  | 028 5949158   | Brentboulevard 99 | Sorée                  |         7752 | Belgium   |
|  2 |        77260 | Cynthia     | Claessens  | dyildirim@example.com | (068)-9861302 | Petraring 62      | Halma                  |         8000 | Belgium   |
|  3 |        20559 | Kevin       | De Wolf    | epauwels@example.org  | (067) 4344408 | Wernerhof 3       | Marchovelette          |         9885 | Belgium   |
|  4 |        42479 | Marie       

In [ ]:
print(pd.read_sql('SELECT TOP 5 * FROM cars', conn).to_markdown())

|    |   ProductID | Brand         | Model                |   Year |   Price |
|---:|------------:|:--------------|:---------------------|-------:|--------:|
|  0 |       22195 | Volvo         | XC90                 |   2018 | 7268.46 |
|  1 |        3045 | Lexus         | LS                   |   2012 | 5610.41 |
|  2 |       12960 | Dodge         | Ram 2500 Regular Cab |   2003 | 7395.68 |
|  3 |       36588 | Mercedes-Benz | 500 E                |   1992 | 2084.09 |
|  4 |       28661 | Hyundai       | Elantra              |   2020 | 5816.77 |


In [ ]:
# Display the first 5 rows of the sales table
conn = engine.connect()
print(pd.read_sql('SELECT TOP 5 * FROM carsales', conn).to_markdown())

|    |   SalesID |   CustomerID |   ProductID |   Quantity |   Price |   DiscountPercent |    Total | SalesAgent          | Date       |
|---:|----------:|-------------:|------------:|-----------:|--------:|------------------:|---------:|:--------------------|:-----------|
|  0 |     90857 |        95329 |       52580 |          3 | 7371.89 |                 8 | 19904.1  | Loes Maris          | 2022-06-13 |
|  1 |      3678 |        82371 |       37902 |          2 | 4008.07 |                 6 |  7214.53 | Tina Roelandt       | 2022-12-10 |
|  2 |     93435 |        12878 |        3045 |          3 | 5610.41 |                 4 | 16158    | Jens Sterckx        | 2023-02-16 |
|  3 |      9868 |        93372 |       10953 |          6 | 6741.73 |                10 | 38023.4  | Annette Vermeulen   | 2023-03-10 |
|  4 |     59970 |        17302 |       74690 |          5 | 7901.77 |                 8 | 35953.1  | Amber Claes De Laet | 2023-04-11 |


In [ ]:
with engine.connect() as conn:
   # What is the most expensive car?
   print('Most expensive car:')
   print(pd.read_sql('SELECT * FROM cars ORDER BY Price DESC', conn).head(1))

   # What city has the most sales renenue?
   print('\nCity with most sales:')
   query = '''
   SELECT TOP 1 City, SUM(Total) AS Revenue
   FROM carsales
   INNER JOIN customers ON carsales.CustomerID = customers.CustomerID
   GROUP BY City
   ORDER BY Revenue DESC
   '''
   print(pd.read_sql(query, conn).head(1))

   # Who is the best sales agent?
   print('\nBest sales agent:')
   query = '''
   SELECT SalesAgent, SUM(Total) AS Revenue
   FROM carsales
   GROUP BY SalesAgent
   ORDER BY Revenue DESC
   '''
   print(pd.read_sql(query, conn).head(1))

   # What is the most popular car?
   print('\nMost popular car:')
   query = '''
   SELECT Brand, Model, SUM(Quantity) AS Quantity
   FROM carsales
   INNER JOIN cars ON carsales.ProductID = cars.ProductID
   GROUP BY Brand, Model
   ORDER BY Quantity DESC
   '''
   print(pd.read_sql(query, conn).head(1))



Most expensive car:
   ProductID Brand Model  Year    Price
0      32626  Audi    A8  1997  9849.09

City with most sales:
       City      Revenue
0  Houwaart  638551.0278

Best sales agent:
     SalesAgent      Revenue
0  Bart Peeters  156777.7753

Most popular car:
   Brand                    Model  Quantity
0  Dodge  Grand Caravan Passenger      1695


## Now let's use GPT to generate SQL queries from natural language questions ⬇️
To make this work we do some things:
1. We first check the database and find all the tables
2. Then the system fetches 5 random rows from each table
3. Then we use GPT to generate SQL queries from natural language questions, passing the table details and the rows as context
4. Optionaly we add a function to parse the result and return it back in natual language

In [ ]:
import openai

openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
# Return the table names in the database
def get_table_names():
    with engine.connect() as conn:
        query = '''
        SELECT TABLE_NAME
        FROM INFORMATION_SCHEMA.TABLES
        WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='vectrix-demo'
        '''
        return pd.read_sql(query, conn)['TABLE_NAME'].tolist()


# Get 5 random rows from a table and store them in a dataframe
def get_random_rows(table, n=5):
    with engine.connect() as conn:
        query = f'SELECT TOP {n} * FROM {table} ORDER BY NEWID()'
        return pd.read_sql(query, conn)


# Call get_random_rows() for each table, and store the results as markdown in a list
markdown = []
for table in get_table_names():
    markdown.append(f'### {table}')
    markdown.append(get_random_rows(table).to_markdown())
    markdown.append('\n')

# Join the markdown list into a single string
table_definitions = '\n'.join(markdown)
table_definitions = table_definitions + '\n---\nReturn the TSQL Query for:'

In [ ]:
GPT_MODEL = "gpt-3.5-turbo"

def generate_query(prompt: str, table_definitions: str):
    """Answers a query using GPT"""
    system = "You are an SQL generator that only returns TSQL sequel statements and no natural language. Given the table names, definitions and a prompt."
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": table_definitions+prompt}
    ]
    #print(messages)

    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=messages,
        temperature=0
    )

    response_message = response["choices"][0]["message"]["content"]
    #print(response_message)

    return response_message

def parse_result_in_natural_language(prompt: str, result: str):
    '''
    Parses the result of a query into natural language
    '''
    completion = prompt + '\n' + result
    messages = [
        {"role" : "system", "content" : "You transalte the result of a query and a prompt into natural language."},
        {"role": "user", "content": completion}
    ]
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages = messages,
        temperature=0
    )
    return response["choices"][0]["message"]["content"]


def run_query(prompt: str, return_natural_language: bool = False):
    query = generate_query(prompt, table_definitions)
    with engine.connect() as conn:
        result =  pd.read_sql(query, conn).to_markdown()

    if return_natural_language:
        result = parse_result_in_natural_language(prompt, result)

    return result


## Let's try it out 🤗
As you can see, when setting the function return_result to True, the system returns the result in natural language. This is done by parsing the result and replacing the column names with the column names in the question.

In [ ]:
print(run_query('What is the most expensive car?', return_natural_language=True))

The most expensive car in the given data is an Audi A8 from the year 1997, which costs 9849.09.


In [ ]:
print(run_query('What city has the most sales revenue?'))

|    | City     |   TotalRevenue |
|---:|:---------|---------------:|
|  0 | Houwaart |         638551 |


In [ ]:
print(run_query('Who is the best sales agent ?', return_natural_language=True))

The best sales agent is Bart Peeters.


In [ ]:
print(run_query('What is the most popular car?'))

|    | Brand   | Model                   |   TotalQuantity |
|---:|:--------|:------------------------|----------------:|
|  0 | Dodge   | Grand Caravan Passenger |            1695 |
